In [2]:
import pandas as pd
from scipy.stats import wasserstein_distance
from statistics import mean

### Lettura dati file degli score

In [4]:
#PATH_FILE_SCORE = './tabellaScoreNuova.csv'
PATH_FILE_SCORE = './scoreCsv.csv'

scores = pd.read_csv(PATH_FILE_SCORE)
scores.head(10)


,Utente1,Utente2,ScoreML
0,1_M_1,1_M_10,0.595889
1,1_M_1,1_M_11,0.567831
2,1_M_1,1_M_12,0.550778
3,1_M_1,1_M_13,0.497624
4,1_M_1,1_M_14,0.539765
5,1_M_1,1_M_15,0.570848
6,1_M_1,1_M_2,0.559740
7,1_M_1,1_M_3,0.562043
8,1_M_1,1_M_4,0.590155
9,1_M_1,1_M_5,0.554073


### Aggiunta colonne di utilità

In [5]:
def calcola_id_soggetto_da_utente(u):
    return f'{u.split("_")[0]}{u.split("_")[1]}'

def calcola_num_acquisizione_da_utente(u):
    return u.split('_')[-1]

In [6]:
scores['Primo soggetto'] = scores['Utente1'].apply(lambda u: calcola_id_soggetto_da_utente(u))
scores['Secondo soggetto'] = scores['Utente2'].apply(lambda u: calcola_id_soggetto_da_utente(u))
scores['Numero prima acquisizione'] = scores['Utente1'].apply(lambda u: calcola_num_acquisizione_da_utente(u))
scores['Numero seconda acquisizione'] = scores['Utente2'].apply(lambda u: calcola_num_acquisizione_da_utente(u))

scores.head()

,Utente1,Utente2,ScoreML,Primo soggetto,Secondo soggetto,Numero prima acquisizione,Numero seconda acquisizione
0,1_M_1,1_M_10,0.595889,1M,1M,1,10
1,1_M_1,1_M_11,0.567831,1M,1M,1,11
2,1_M_1,1_M_12,0.550778,1M,1M,1,12
3,1_M_1,1_M_13,0.497624,1M,1M,1,13
4,1_M_1,1_M_14,0.539765,1M,1M,1,14


In [33]:
a = scores[scores['Primo soggetto'] != scores['Secondo soggetto']]
a.ScoreML.max()

0.373056241825316

### Calcolo delle pseudo-labels

In [7]:
def get_set_intra_classe(utente, scores):
    id_soggetto = calcola_id_soggetto_da_utente(utente)
    id_acquisizione = calcola_num_acquisizione_da_utente(utente)
    # Filtro per soggetto
    soggetto_scores = scores[(scores['Primo soggetto'] == id_soggetto) & (scores['Secondo soggetto'] == id_soggetto)]
    # Filtro per acquisizione
    soggetto_scores = soggetto_scores[(soggetto_scores['Numero prima acquisizione'] == id_acquisizione) | (soggetto_scores['Numero seconda acquisizione'] == id_acquisizione)]
    return set(soggetto_scores["ScoreML"])


def get_set_inter_classe(utente, scores):
    id_soggetto = calcola_id_soggetto_da_utente(utente)
    id_acquisizione = calcola_num_acquisizione_da_utente(utente)    
    
    id_soggetto = calcola_id_soggetto_da_utente(utente)
    id_acquisizione = calcola_num_acquisizione_da_utente(utente)
    # Filtro per soggetto
    soggetto_scores = scores[((scores['Primo soggetto'] == id_soggetto) & (scores['Secondo soggetto'] != id_soggetto)) | ((scores['Primo soggetto'] != id_soggetto) & (scores['Secondo soggetto'] == id_soggetto))]
    # Filtro per acquisizione
    soggetto_scores = soggetto_scores[(soggetto_scores['Utente1'] == utente) | (soggetto_scores['Utente2'] == utente)]
    return set(soggetto_scores["ScoreML"])
    

In [8]:
TRADE_OFF_PARAMETER = 1

lista_utenti = list(set(scores.Utente1.unique()) | set(scores.Utente2.unique()))
lista_utenti.sort()

In [9]:
df_labels = pd.DataFrame(columns=['Acquisizione', 'Label']) 

for utente in lista_utenti:
    set_intra_classe = get_set_intra_classe(utente, scores)
    QxP = mean(set_intra_classe)

    set_inter_classe = get_set_inter_classe(utente, scores)
    list_inter_classe = list(set_inter_classe)
    list_inter_classe.sort(reverse=True)
    list_confronto_intra_classe = list_inter_classe[0: len(set_intra_classe)]
    QxN = wasserstein_distance(list(set_intra_classe), list_confronto_intra_classe)

    label = QxP + TRADE_OFF_PARAMETER * QxN

    row_acquisizione = pd.DataFrame.from_dict({'Acquisizione': utente, 'Label': label}, orient='index').T
    df_labels = pd.concat([df_labels, row_acquisizione])


### Normalizzazione delle labels tra 0 e 100

In [10]:
min_label = df_labels.Label.min()
max_label = df_labels.Label.max()
df_labels['Label_norm'] = df_labels['Label'].apply(lambda l: 100 * (l - min_label)/(max_label - min_label))
df_labels = df_labels.reset_index(drop=True)

In [18]:
df_labels.sort_values(by=['Label'])

,Acquisizione,Label,Label_norm
121,47_F_5,0.252351,0.000000
161,58_M_9,0.377873,14.123512
185,86_M_12,0.407557,17.463577
184,86_M_11,0.435625,20.621737
182,86_M_1,0.463319,23.737839
...,...,...,...
103,41_M_20,1.113939,96.944848
108,41_M_7,1.120439,97.676251
100,41_M_18,1.121578,97.804376
109,41_M_8,1.129497,98.695452


### Analisi su singola acquisizione

In [28]:
acquisizioneAnalisi = '33_F_3'
test_df = scores[(scores.Utente1 == acquisizioneAnalisi) | (scores.Utente2 == acquisizioneAnalisi)]
test_df.sort_values(by='ScoreML').style

,Utente1,Utente2,ScoreML,Primo soggetto,Secondo soggetto,Numero prima acquisizione,Numero seconda acquisizione
11682,33_F_3,58_M_5,0.148523,33F,58M,3,5
11697,33_F_3,75_M_19,0.152720,33F,75M,3,19
11665,33_F_3,51_M_15,0.153767,33F,51M,3,15
11683,33_F_3,58_M_6,0.155746,33F,58M,3,6
11662,33_F_3,51_M_12,0.156525,33F,51M,3,12
11663,33_F_3,51_M_13,0.157474,33F,51M,3,13
5722,27_F_1,33_F_3,0.158125,27F,33F,1,3
8212,27_F_5,33_F_3,0.159752,27F,33F,5,3
7570,27_F_2,33_F_3,0.160914,27F,33F,2,3
11696,33_F_3,75_M_18,0.161191,33F,75M,3,18


In [23]:
df_labels.to_csv('./tabellaPseudoLabelsCompleta.csv', index=False)